# Finesse Tutorial: The benefits of the cavities inside the optical layout of an advanced detetcor 
Andreas Freise, Alina Soflau, Yashwant Bothra, Enzo Tapia 

Nikhef, 10.10.2025

This notenook was used in the IMPRS School "Gravity at the Extreme: from Theory to Observation", October 2025.

# Sensitivity of advanced gravitational-wave detectors

In this notebook we will work with plots that are similar to the famous sensitivity plots you will often see. For this we will use roughly similar noises as the current detectors such as Advanced LIGO and Advanced Virgo. Our results will not be exactly the same as the official plots, however this should give you an appreciation of the steps involved in how to compute the sensitivity.

**After working though this notebook you will be able to**:

* compute the response of a GW detector to gravitational-wave signals
* compute the quantum-noise limited sensitivity
* read and explain a sensitivity curve

**This notebook assumes you know:**
* what a transfer function is
* what a signal-to-noise ratio means
* what amplitude and power spectral densities are
* how to run and create Finesse (version 3) 

# Goals of this notebook

The majority of this notebook is dedicated to explaining various concepts and modeling techniques in Finesse. Go through them in order to understand how modeling is done in Finesse. These concepts are essential for you to complete the Tasks and to interpret their results.

**To-do's:**

1. In this notebook, there are several tasks that you need to complete using Finesse. These tasks are related to the topic of how cavities in the optical layout change the performance of the detector.

2. After completing the work, you will give a short presentation (5–10 minutes) on your completed tasks. As you work through the tasks and question sets, it would be helpful to populate your slides simultaneously.

### 1. Overview 

Current gravitational-wave detectors are based on the Michelson interferometer, the simplest configuration capable of measuring tiny differential arm-length changes. Over time, this basic topology has been extended and refined through several improvements. 

In this notebook, we compare the Michelson setup with successive optical designs to see how each step changes the interferometer’s transfer functions and sensitivity. 

The configurations under study are: 

1) Michelson Interferometer (MI). This is the simplest layout: a beam splitter and two perpendicular arms with mirrors at the ends.
2) Fabry-Perot Michelson Interferometer (FPMI). Here we have a partially transmitting mirror at the start of each arm, so each arm becomes a resonant optical cavity, also known as Fabry-Perot (FP) cavity. As a result, the light travels a longer optical path inside the arms. 
3) Power Recycling Fabry-Perot Michelson Interferometer (PRFPMI). With respect to the previous layout, here an additional mirror, the Power Recycling Mirror (PRM) is placed between the laser and the beam splitter. This mirror forms a resonant cavity with the rest of interferometer, called Power Recycling Cavity (PRC). Since a large part of the incident power is reflected by the interferometer towards the laser, the PRM allows for increased
circulating power, without the need to boost the input power. This technique, known as power recycling, effectively "recycles" light that would otherwise be lost.
4) Dual Recycling Fabry-Perot Michelson Interferometer (DRFPMI). In addition to Fabry Perot and Power Recycling cavities,  we have the Signal Recycling Mirror (SRM), placed between the beam splitter and the photodiode. It also forms an optical resonator with the interferometer, called Signal Recycling Cavity (SRC). By carefully adjusting the SRM’s position, the resonance of this cavity can be tuned to specific frequencies, enhancing the detector’s sensitivity to specific types of gravitational waves. Via the SRM we can also narrow or broaden the frequency range of the detector.

### 2. From noise sources to strain sensitivity

For any noise source that affects the detector output, we can estimate its impact on strain sensitivity by comparing how strongly the interferometer responds to that noise versus to a real gravitational-wave signal.

We define:  

- **Hs(f)** — signal transfer function: response of the detector output power to a gravitational-wave strain [W/h]  
- **Hn(f)** — noise transfer function: response of the detector output power to a specific technical noise source (e.g. laser frequency noise). Units: W / (unit of noise); for laser frequency noise, units are W / Hz.

- **Nn(f)** — amplitude spectral density (ASD) of that noise source [$\text{unit}/\sqrt{\text{Hz}}$]

The corresponding **strain-equivalent noise**  expresses how strongly a given noise source mimics a real gravitational-wave signal at the detector output, and is given by:  

$$
S_h(f) = \frac{|H_n(f)|}{|H_s(f)|} \, N_n(f)
$$

### 3. Noise sources: quantum noise 

A complex instrument such as a GW detector can be affected by many noise sources that can mask or mimic a gravitational wave signal. In this notebook, we look at a few representative examples: 

**Quantum noise** is an umbrella term used to refer to multiple complicated effects in the interferometer. However, they are all a direct consequence of the photonic nature of the light field, their associated Poissonian statistics, and the Heisenberg uncertainty principle. The details are unimportant for our modeling tasks, but the interested reader may find this [review article](https://arxiv.org/abs/1203.1706) informative.

In Finesse the `qnoised` detector can be used to directly compute the quantum noise amplitude spectral density at the output port of an interferometer.

The output of `qn` has the form $N_q(f) |H_p(f)|$, where $N_q$ is the amplitude spectral density of the quantum noise and $H_p$ the transfer function from quantum noise to the output port, so 
we can then directly combine this output with the transfer function computed in task 1 to get the *Quantum Noise Limited Sensitivity* of the interferometer:
$$S_q(f) = \frac{\mathtt{out[`qn`]}}{|H_s(f)|}$$

### 4. Noise sources: laser frequency noise

Interferometers are also affected by **laser frequency noise**: the laser’s optical frequency is not perfectly constant in time, but oscillates slightly around its nominal (central) value. If the laser normally emits light at frequency $\nu_0$, then applying a frequency modulation makes it vary as

$$ \nu(t)= 
\nu_0 + \delta \nu(t)= 
\nu_0+  {A}  \cdot \sin (2 \pi f_{\text{sig}}t) $$ 

where A is the modulation amplitude.

A change $ \nu(t)= \nu_0+ \delta \nu(t) $ 
corresponds to a change in phase of the input field: 
$$ E_{\mathrm{in}}(t) = 
E_0 e^{i(2\pi \nu_0 t + \delta \phi(t))}. $$

Here 
$$
\delta \phi(t)= 2 \pi A \int \sin (2 \pi f_{\text{sig}}t) dt = 
- \dfrac{A}{ f_{\text{sig}}} \cos (2 \pi f_{\text{sig}}t)
$$  

Because the interferometer output is determined by the interference between the beams returning from its two arms, fluctuations in the laser frequency alter the interference condition, and thus the detected signal.

In [ ]:
# install pypi wheel only on google colab
!env | grep -q 'colab' && pip install finesse || echo 'Not on google colab, assuming finesse already installed'

In [ ]:
# Importing key python packages 
import matplotlib.pyplot as plt
import numpy as np

# Importing Finesse and one specific action
import finesse
from finesse.analysis.actions import Xaxis
finesse.init_plotting()

# Introductory task: role of cavities

Optical cavities are the fundamental building blocks of gravitational-wave (GW) detectors. In this section, we will briefly study their characteristics and explain their role. Understanding cavities is essential, as most of the interferometer configurations we will encounter in this tutorial rely on multiple cavities to enhance the detector’s sensitivity over a given frequency bandwidth.

For simplicity, we will consider a simple Fabry–Perot cavity. In this model, the Input Test Mass (ITM) and End Test Mass (ETM) form a cavity. The main purpose of these arm cavities is to increase the circulating power inside the interferometer arms and thereby amplify the GW signal. A passing gravitational wave changes the optical path length between the two test masses, producing sidebands on the laser light. The cavity acts as a resonant amplifier, enhancing these sidebands and making the signal measurable at the output port. 

In sumamry:
1. Power Build-Up: Cavities resonate the carrier light, increasing the effective power inside the arms by a factor equal to the cavity finesse. This reduces shot noise and improves strain sensitivity.
2. Signal amplification: By storing light for many round trips, the cavity amplifies the phase shift caused by GWs, effectively increasing the interferometer’s arm length (folding of arms).
3. Frequency Shaping: Cavities act as filters, enhancing sensitivity in certain frequency bands (e.g., arm cavities set the cavity pole, which determines the frequency response).
4. In advanced configurations, power and signal recycling cavities are added to further improve sensitivity and tailor the detector’s response.

In [ ]:
## Simple cavity model

cavm = finesse.Model()
cavm.parse(
    """
    l LI P=125
    s s1 LI.p1 ITMX.p1 L=5

    # Simple Fabry-Perot Cavity
    m ITMX T=0.014 L=37.5u 
    s Lx ITMX.p2 ETMX.p1 L=3995
    m ETMX T=0.01 L=37.5u phi=3m
    """
)

## Studying Cavity Resonance
cavstudy = cavm.deepcopy()

## Adding complementary detectors to the model
cavstudy.parse(
    """
    pd pdtrans ETMX.p2.o #detector to check power transmitted
    pd pdcirc ETMX.p1.o #detector to check circulating power
    pd pdref ITMX.p1.o #detector to check reflected laser power
    """ 
)

## Scanning the microscopic tuning of the cavity to see resonances (or Free spectral range)
scan = cavstudy.run(Xaxis("ETMX.phi", 'lin', start=-180, stop=180, steps=200))

## Plots
plt.plot(scan.x[0], scan['pdtrans'], label='Power Transmitted')
plt.plot(scan.x[0], scan['pdcirc'], label='Power Circulating')
plt.plot(scan.x[0], scan['pdref'], label='Power Reflected')
plt.yscale('log')
plt.legend()
plt.show()

Imagine the cavity as a black box. This black box has a certain reflectivity and transmissivity, both of which depend on the resonance condition inside the cavity. If the cavity is resonant for the given laser frequency, the circulating power inside the cavity increases drastically, and correspondingly, the transmitted power also increases. At the same time, the reflected power drops. All of this follows from the principles of energy conservation and the resonance condition of the cavity.

From the plot above, we can see that the cavity is resonant for the laser frequency and is amplifying the circulating power. One can think of a cavity in several ways. For example, based on the plot, it can be thought of as an energy reservoir, where electromagnetic energy is stored through constructive interference of light.

In [ ]:
## Calculating few important characteristics of a cavity

# Free spectral range
FSR = 2.99e8/(2*cavstudy.Lx.L)
print(f"Free spectral range of the cavity = {FSR:.1f}")

# Finesse of our cavity
r1 = np.sqrt(1-cavstudy.ITMX.T-cavstudy.ITMX.L)
r2 = np.sqrt(1-cavstudy.ETMX.T-cavstudy.ETMX.L)
F = np.pi*np.sqrt(r1*r2)/(1-(r1*r2))

print(f"Finesse of the cavity = {F:.1f}")

# Cavity pole
fpole = FSR/(2*F)

print(f"Cavity pole = {fpole:.1f}")


In [ ]:
## Frequency response of a simple FP cavity for laser frequency noise
fresponse = cavm.deepcopy()

# Adding code to study laser frequency fluctuations at the output of the cavity
fresponse.parse(
    """
    fsig(1)
    sgen sig1 LI.frq 1 0 
    pd1 TF ETMX.p2.o f=fsig.f
    """
)

# Scanning over a given frequency range
out1 = fresponse.run(Xaxis('fsig.f', 'log', 5, 1e4, 200))

# Plot
plt.loglog(out1.x[0], abs(out1['TF']), label="Laser frequency noise TF")
plt.axvline(x=fpole, color='black', linestyle='--', linewidth=1, label='Cavity pole')
plt.legend();

The frequency response of a simple cavity depends strongly on its characteristics—such as mirror transmissions, losses, free spectral range (FSR), and finesse. This response tells us how the cavity behaves dynamically when subjected to frequency or length fluctuations.

For laser frequency modulations with f << cavity pole, the cavity is able to follow the modulation perfectly, as seen from the transfer function above. However, for f >> cavity pole the cavity cannot respond quickly to the fluctuations, and a roll-off appears beyond the cavity pole.

The cavity pole frequency and the photon storage time are inversely related. A high-finesse cavity has a long photon storage time but a narrower frequency bandwidth. A long storage time allows the cavity to build up circulating power and to enhance the sidebands produced by gravitational waves. This is why it is crucial for GW detectors to use high-finesse arm cavities—they increase the signal response within the sensitive band.

One can think of a cavity as a frequency filter, due to its finite photon storage time. It acts like a low-pass filter for any perturbation—whether it originates from laser frequency noise, mirror motion, or GW-induced sidebands.

# Task 1. Build the interferometer configurations

Here you will complete the optical layouts for four interferometer configurations of increasing complexity, by adding the missing port connections. First we define the four build Finesse models for the four configaritions, by describing their optical layouts in KatScript.

You should always draw your own sketch with pen and paper of any optical layout you want model. Below are four diagrams to illutsrate the configurations we will be using.

<img src="figures/Michelson_ITF.png" style="background: rgba(255,255,255,1); width: 45%;"/>
<img src="figures/FP_Michelson_ITF.png" style="background: rgba(255,255,255,1); width: 45%;"/>

<img src="figures/PRFPMI_DRFPMI.png" style="width: 90%; background: rgba(255,255,255,1)"/>

In [ ]:
# Modelling the basic Michelson interferometer

# Creating the Finesse model object
ifo_MI = finesse.Model()

# Parsing a string of 'KatScript' to define the interferometer
ifo_MI.parse(
""" 
l LI P=125
s s1 LI.p1 bs1.p1 L=5

# Central beamsplitter
bs bs1 R=0.5 T=0.5 phi=0 alpha=45

# X-arm
s Lx bs1.p3 ETMX.p1 L=3995+4.5
m ETMX T=0 L=37.5u phi=89.997

# Y-arm
s Ly bs1.p2 ETMY.p1 L=3995+4.45
m ETMY T=0 L=37.5u phi=3m

# These commands give the mirrors a mass
# by default they are infinitely heavy
free_mass ETMX_sus ETMX.mech mass=40
free_mass ETMY_sus ETMY.mech mass=40
""") 

In [ ]:
# Similarly we create a model for a Michelson with optical cavities in the arms

ifo_FPMI = finesse.Model()
ifo_FPMI.parse(
""" 

# complete the code here               

# X-arm
s sx bs1.p3 ITMX.p1 L=4.5
m ITMX T=0.014 L=37.5u phi=89.997
s Lx ITMX.p2 ETMX.p1 L=3995
m ETMX T=0 L=37.5u phi=89.997

# Y-arm
s sy bs1.p2 ITMY.p1 L=4.45
m ITMY T=0.014 L=37.5u phi=3m

# complete the code here

""") 

In [ ]:
# Now, the 'power-recycled' version of the above.

ifo_PRFPMI = finesse.Model()
ifo_PRFPMI.parse(
""" 

# complete the code here

# Power recycling mirror
m PRM T=0.03 L=37.5u phi=90
s prc PRM.p2 bs1.p1 L=53

# complete the code here

""") 

In [ ]:
# And finally the full dual-recycled version, which represents the optical 
# layout of the Advanced LIGO and Advanced Virgo detectors

ifo_DRFPMI = finesse.Model()
ifo_DRFPMI.parse(
""" 
# complete the code here

# Signal recycling mirror
s src bs1.p4 SRM.p1 L=50.525
m SRM T=0.2 L=37.5u phi=-90

# complete the code here

""") 

# Task 2. Compute and compare the gravitational-wave signal transfer functions


We now get to the stage we can model how a gravitational-wave signal would appear in our detector. The gravitational wave will modulate the phase of the light travelling along a space.

Questions:

- Why do we apply opposite phase modulations (0° and 180°) to the two arms? Hint: think about how a gravitational wave stretches one arm while squeezing the other.

- What does a high/low transfer function mean? Recall that  $\text{H}_s(f)$ is: 

$$

H_s (f) =\dfrac{ \text{output signal} \, [W]} {\text{GW strain h}}

$$. 


- Why is the Michelson curve almost flat?

- Starting from the Michelson, each time we add more cavities (Fabry–Perot arms, Power Recycling), the transfer function amplitude becomes larger. Why? Hint: a cavity that stores light for many round trips; a small change in arm length creates a larger phase change. 

- But when we finally add Signal Recycling (Dual-Recycled case), the curve goes down. Does this mean the detector got worse? Hint: notice that yes, the curve is lower, but it stays flat over a wider range of frequencies compared to the Power-Recycled case.

In [ ]:
# The frequency vector we'll use for plotting transfer funcctions and
# sensitivty curves through this notebook
fstart = 5
fstop = 1e4
Npoints = 400

## MI:
ifo_MI_base = ifo_MI.deepcopy()
ifo_MI_base.parse(f"""
fsig(1)

sgen sig1 Ly.h 1 0
sgen sig2 Lx.h 1 180

pd1 TF bs1.p4.o f=fsig.f
""")
out_MI = ifo_MI_base.run(Xaxis('fsig.f', 'log', fstart, fstop, Npoints))
# We store the frequency and magnitude of the transfer function [W/h]
# (Usually) we need to store the magnitude and phase of the transfer 
# functions but we only need the magnitude for these examples.)
f, Hs_MI = out_MI.x1, np.abs(out_MI['TF'])

## FPMI:
ifo_FPMI_base = ifo_FPMI.deepcopy()
# ...

In [ ]:
plt.loglog(f, Hs_DRFPMI, label='DRFPMI')
plt.loglog(f, Hs_PRFPMI, '--', label='PRFPMI')
plt.loglog(f, Hs_FPMI, '--', label='FPMI')
plt.loglog(f, Hs_MI, '--', label='MI')
plt.title("Transfer function for GW signal to main output")
plt.legend();

# Task 3. Compute and compare the Quantum Noise Limited Sensitivity (QNLS)

Compute the QNLS for all configurations and explain the shapes of the curves. Hint: At low frequencies, mirror motion dominates (radiation pressure noise). At high frequencies, light’s random photon statistics dominate (shot noise).  Shot noise is typically improved by increasing power. However, increased power means more radiation pressure noise.

Questions: 

- Why do all the curves go up again at very high frequencies?

- Which interferometer configuration would perform best in a real gravitational-wave detector, and why?
Hint: Think about which setup offers the best broadband sensitivity, not just the lowest noise at one frequency.


In [ ]:
# Code hint for the students

## MI:
ifo_MI_qn = ifo_MI.deepcopy()
ifo_MI_qn.parse(f"""
fsig(1)
# compute the quantum-noise in the detector output
qnoised qn bs1.p4.o
""")
out_MI_qn = ifo_MI_qn.run(Xaxis('fsig.f', 'log', fstart, fstop, Npoints))

S_q_MI = ...

## FPMI:
ifo_FPMI_qn = ifo_FPMI.deepcopy()
# ...

In [ ]:
plt.loglog(f, S_q_DRFPMI, label="DRFPMI")
plt.loglog(f, S_q_PRFPMI, label="PRFPMI")
plt.loglog(f, S_q_FPMI, label="FPMI")
plt.loglog(f, S_q_MI, label="MI")
plt.xlabel("Frequency [Hz]")
plt.ylabel("Sensitivity [$h/\\sqrt{Hz}$]")
plt.legend()
plt.tight_layout();

# Task 4. Compute and compare strain sensitivity spectra due to laser frequency noise

You can define a function to compute transfer functions automatically.

The function should: 

- create a copy of the interferometer model (to avoid modifying the original),

- inject a small signal modulation at the chosen node, (e.g. `LI.frq` for laser frequency noise, or `Ly.h`, `Lx,h` for gravitational-wave strain),

- measure the change in optical power at the user-defined detector port,

- return the transfer function as a function of frequency. 

Below, we provide simple analytic models for laser frequency noise.

In [ ]:
# Computer transfer function from a specific input node to a photo detector in the 
# main interferometer outout

def get_TF(model, input_node, output_node):
    ifo = model.deepcopy()
    ifo.parse(f"""
    fsig(1)
    sgen sig1 {input_node} 1 0
    #pd1 TF SRM.p2.o f=fsig.f
    pd1 TF {output_node} f=fsig.f
    """)
    return ifo.run(Xaxis('fsig.f', 'log', fstart, fstop, Npoints))

In [ ]:
# Simple (but fake) model of frequency noise
def laser_freq_noise_ASD(f):
    f = np.array(f)
    return 1e-2 / np.abs(f) * (1 + 0.1 * np.random.randn(f.size)) # pre-stabilisaed
        
N_freq = laser_freq_noise_ASD(f)

plt.figure()
plt.loglog(f, N_freq, label="free-runing laser freq noise")
plt.legend()
plt.xlabel("Frequency [Hz]");


In [ ]:
out_MI = get_TF(ifo_MI, 'LI.frq', 'bs1.p4.o')
Hn_MI = np.abs(out_MI['TF'])
# ...

In [ ]:
plt.figure()
plt.loglog(f, Hn_MI, label="Freq noise TF MI", alpha=0.5, linewidth=3)
plt.loglog(f, Hn_FPMI, '--', label="Freq noise TF FPMI", alpha=0.5, linewidth=3)
plt.loglog(f, Hn_PRFPMI, '--', label="Freq noise TF PRFPMI", alpha=0.5, linewidth=3)
plt.loglog(f, Hn_DRFPMI, '--', label="Freq noise TF DRFPMI", alpha=0.5, linewidth=3)
plt.legend();

In [ ]:
# The same equation as above but inverted to compute the frequency noise from 
S_freq_MI = 
S_freq_FPMI = 
S_freq_PRFPMI = 
S_freq_DRFPMI = 